# DIY Covid-19 Dashboard

This is a dashboard of This is a dashboard showing the distribution of COVID-19 death data based on age in England.


First, we should import libararies we need

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

We have put the .json file and the dashboard together, and our dashboard needs to read the .json file

In [2]:
# Load JSON files and store the raw data in some variable.
with open("deathagedistribution.json","rt") as INFILE:
    data=json.load(INFILE)

FileNotFoundError: [Errno 2] No such file or directory: 'deathagedistribution.json'

## Wrangle the data

We need a function to process the raw data to make it into a table form. And this function can also used to process the raw data we get from api.

In [ ]:
def wrangle_data(rawdata):
    # Parameters: rawdata - data from json file or API call. Returns a dataframe.
    
    # First, we need to define the empty form
    df=pd.DataFrame(index=range(0,100), columns=['One', 'Two'])
    datadic=data['data'][0]
    malesdeaths=datadic['maledeaths']
    femaledeaths=datadic['femaledeaths']
    ageranges=[x['age'] for x in malesdeaths]
    
    # get every agerange's min age
    def min_age(agerange):
        agerange=agerange.replace('+','')
        start=agerange.split('_')[0]
        return int(start)
    
    # sort agerange
    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges,columns=['maledeaths','femaledeaths'])
    
    # We fill the data into the form
    for entry in malesdeaths:
        ageband=entry['age']
        age_df.loc[ageband,'maledeaths']=entry['value']
    for entry in femaledeaths:
        ageband=entry['age']
        age_df.loc[ageband,'femaledeaths']=entry['value']
    age_df['total']=age_df['maledeaths']+age_df['femaledeaths']
    return age_df
"""putting the wrangling code into a function allows you to call it again after refreshing 
the data through the API. You should call the function directly on the JSON data when the 
dashboard starts, by including the call in the cell as below:"""
df=wrangle_data(data) # df is the dataframe for plotting

## Download current data

Give users an option to refresh the dataset - a "update data" button. And once we update the dataset, the file will be updated too.

In [ ]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the JSON file. """
    filter = [
        'areaType=nation',
        'areaName=England'
    ]
    structure = {
        "maledeaths":"maleDeaths28Days",
        "femaledeaths":"femaleDeaths28Days"
    }
    api = Cov19API(filters=filter,structure=structure)
    deathagedistribution28days=api.get_json()
    return deathagedistribution28days # return data read from the API

In [ ]:
def api_button_callback(button):
    # Get fresh data from the API.
    apidata=access_api()
    # wrangle the data and overwrite 
    global df
    df=wrangle_data(apidata)
    # update raw data yo json file
    with open("deathagedistribution.json","wt") as OUTF:
        json.dump(apidata,OUTF)
    refresh_graph()
    # refresh graph
    apibutton.icon="check"
    apibutton.disabled=True
    # after clicking the button, the button will be "Unavailable"
    
# set button style    
apibutton=wdg.Button(
    description='Update data', 
    disabled=False,
    button_style='', 
    tooltip="Click to download current Public Health England data",
    icon='download'
)


apibutton.on_click(api_button_callback) 

# we can display our button

display(apibutton)



## Graphs and Analysis

We can plot our graph with processed data and add a controller.

In [ ]:
def plot(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() 
    else:
        # if the user has not selected any column, print a message 
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

agecols=wdg.SelectMultiple(
    options=['maledeaths', 'femaledeaths', 'total'], # options available
    value=['maledeaths', 'femaledeaths'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False
)

def refresh_graph():
    """ We regenerate the graph after acquiring new data. """
    garph=wdg.interactive_output(plot,{'graphcolumns': agecols})
    display(agecols,graph)
    
    
graph=wdg.interactive_output(plot, {'graphcolumns': agecols})
    
display(agecols, graph)